In [ ]:
#purpose is to generate images in the steps below, first collecting some files
'''
Paths and file needs:
*imglams and spitzer_conversions are excel files, right now I have it so you need to put it as same directory as your code (but could later maybe just give it a path to go to - would be smarter)
*paths to images and data in general
'''

#just to check python version - should be 3.7.4
from platform import python_version
print(python_version())

#importing libraries
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel, Box2DKernel
from astropy.nddata import Cutout2D
from astropy.wcs import WCS

import glob
import itertools
import matplotlib 
matplotlib.use('Agg') #invokved b/c just plain matplotlib was insufficient
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys



In [ ]:
# #finding the path to every fits images in a directory
def im_name_finder(path, file_type):
    #Using glob (it's a unix command similar to ls)
    #WARNING: using recursive=True...depending how many images you use this could be very slow, it's recommended not to have too many subfolders
    #if needed, some example code is commented towards the latter half of this code that could help make an alternative
    all_names = glob.glob(path, recursive=True)

    #IMPORTANT: Using "fit" here because it is inclusive of both fits and FIT...some files end in "FIT" and need to be included
    #using s.lower() include uppercase names
    im_names = [s for s in all_names if 'fit' in s.lower()]

    return im_names


'''now convolve my image with a PSF of the image we're projecting ONTO
an approx PSF can be found by assuming a 2D Gaussian func with a width (a FWHM) of the diffrac limit
that is the st dev of the Gaussian is about the st dev is about = lambda/D
a list of PSFs are found on https://docs.astropy.org/en/stable/convolution/kernels.html

Notes:
FIRST: always must convert hdu1_pixtorad to radians! It's inconsistent otherwise, and lambda/D is generally in radians

what we're using for the gaussian width is the FWHM, not the radius of the first ring of the diffraction pattern,
so it's 1.2 not 1.22 times lambda/D

D is 85 cm for spitzer
D is 2.4 m for hubble
'''

def im_conv(D, hdu_pix_torad, hdu_dat, lam, kern):
    #gaussian kernel
    if kern == 'gauss':
        #update: usually cannot find wavelength but these headers are well-labeled    
        #finding angular resolution...the FWHM of our Gaussian PSF
        res = 1.2 * lam / D         #resolution in radians
        res = res / hdu_pix_torad        #so converting to pixels

        #finding PSF and then calculating the convolution of our image and the PSF of the image we're projecting onto
        kernel = Gaussian2DKernel(res)

    #box kernel
    if kern == 'box':
        kernel = Box2DKernel(16.)

    hdu_conv = convolve(hdu_dat, kernel)
    return hdu_conv

# In[27]:

#setting up a new fits file to be saved and viewed in DS9
#primarily to save the image we reprojected, but can also be used to save the convolved images
def fits_saver(array, wcs_header, name, save_path):
    '''
    array is a 2d array of data - could be from reprojecting one image onto another or from convolution
    wcs_header is a header containing the wcs coords of the image that we projected onto or of the orig image (if from the convolution)
    name is the path to some image you're using. It will get string split at the / character, and the func only takes the last element of that splitting
    save_path is the folder you want to save to...recommended to also add something to the start of the images names to make it clear what you did to them (e.g. 'Regridded/regrid_')
    '''

    #creating a new file and adding the reprojected array of data as well as the WCS that we projected onto
    hdu_new = fits.PrimaryHDU(array, header=wcs_header)
    hdul = fits.HDUList([hdu_new])
    
    #saving the file
    if name.find('FIT') == -1: #needed if file end incorrect
        new_filename = name.split('/')[-1]  #grabs the file name we were using from before
        hdul.writeto(save_path+new_filename, overwrite=True)
    else:
        name_fixfit = name[:-3] + 'fits'
        new_filename = name_fixfit.split('/')[-1]  #grabs the file name we were using from before
        hdul.writeto(save_path+new_filename, overwrite=True)
        
    return (save_path+new_filename)

This code uses cross correlation
Background paper examples
https://articles.adsabs.harvard.edu/pdf/1992ApJ...392..145R or https://iopscience.iop.org/article/10.1086/517493/pdf

Example stack overflow references...
https://stackoverflow.com/questions/24768222/how-to-detect-a-shift-between-images (the best)
https://stackoverflow.com/questions/61114057/how-to-estimate-motion-with-ftt-and-cross-correlation (not bad)
https://stackoverflow.com/questions/189943/how-can-i-quantify-difference-between-two-images (good conceptual explanation)

In [ ]:
#here we try to grab a 1.64 mic fits image with minimal 

path = '../continuum_subtract/nocont*164*.fits' # #using ** will grab all files even in subdirectories WARNING takes longer
im_names_n2071 = sorted(im_name_finder(path, 'fit')) #im_finder is basically glob.glob
im_names_n2071 = [i.replace('\\', '/') for i in im_names_n2071]
print(im_names_n2071)

In [ ]:
hdu_list = [fits.open(i) for i in im_names_n2071]

#initializing some lists to be used
hdu_data_list = []
hdu_header_list = []

count = 0
for hdu_data in hdu_list:   
    #reading in data for general use  and header for wcs
    #converting by times by flam * bw from e-/sec...should get units of erg/cm^2/sec as above
    
    #needed because the second image in this list is negative...
    if count == 1:
        sign = -1
    else:
        sign = 1
    hdu_data_list.append(sign * hdu_data[0].data) # * hdu_list[0].header['PHOTFLAM'] * hdu_list[0].header['PHOTBW'])
    hdu_header_list.append(hdu_data[0].header)
    
    count+=1

In [ ]:
#our plotting function
def implot(data, w, wcscond, vmax_p):
    fig = plt.figure()
    
    if  wcscond == True:
        fig.add_subplot(111, projection=w)
    else:
        fig.add_subplot(111)
    
    #for christmas turn on GnRd
    #plt.cm.get_cmap('Blues', 6) is another option
    #can also use RdBu...
    #otherwise just use plt.cm.viridis b/c it works
    plt.imshow(data, origin='lower', cmap=plt.cm.viridis, vmin =0, vmax=vmax_p)
    plt.xlabel('RA')
    plt.ylabel('Dec')


In [ ]:
%matplotlib inline

# Reading in a region file + indexing for cutouts
Seems like doing it with cutout2d for once worked better than doing it by hand...so let's just go with it.

Will help to then overlay this on the difference image

In [ ]:
#trying this with cutout2d
# https://docs.astropy.org/en/stable/nddata/utils.html#cutout-images
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import SkyCoord, FK5, ICRS
from photutils.aperture import EllipticalAperture

#trying it by hand

#known pixel size
hst_pixsize = 0.12825 #arcsec

# f = open('moving_blobs.reg', 'r')
# f1 = open('precession_inflections.reg', 'r')
f1 = open('epoch2_361c_byhand_ellipses.reg', 'r')
file1_output = []

#looping through file 
for line in f1:
    file1_output.append(line)
f1.close()

#fix file contents
#contents are ra, dec, width, height, rotation
file1_output = [i[8:-2] for i in file1_output[3:]]

#next step is for image, loop through all regions and make a list of region properties
ra_pix_list = []
dec_pix_list = []
rad_a_list = []
rad_b_list = []
rotation_list = []

#coordinate details
split_params1 = [i.split(',') for i in file1_output[:-1]]
ra_hms_list = [ra1[0].split(':')[0]+'h' + ra1[0].split(':')[1]+'m' + ra1[0].split(':')[2]+'s' for ra1 in split_params1]
dec_dms_list = [dec1[1].split(':')[0]+'d' + dec1[1].split(':')[1]+'m' + dec1[1].split(':')[2]+'s' for dec1 in split_params1]
pos_list = [SkyCoord(ra+' '+dec, frame=FK5, unit=(u.hourangle, u.deg)) for ra, dec in zip(ra_hms_list, dec_dms_list)]

#this isn't working for some reason...skycoord_to_pixel doesn't seem to like units? not sure
ra_pix_list = [skycoord_to_pixel(i, wcs=WCS(hdu_header_list[0]))[0] for i in pos_list]
dec_pix_list = [skycoord_to_pixel(i, wcs=WCS(hdu_header_list[0]))[1] for i in pos_list]

#aperture size details
rad_a_list = [[1./hst_pixsize * float(rad_a1[2][:-1])] for rad_a1 in split_params1]
rad_b_list = [[1./hst_pixsize * float(rad_b1[2][:-1])] for rad_b1 in split_params1]
rotation_list = [[float(rot1[4])] for rot1 in split_params1]

# Plotting markers on image for reference

Ideally, we should see something like a zig-zig or a parallelogram...

Also check order of points we ended up with

Looks like the source star (361C) is labeled as ind = 3

In [ ]:
plt.imshow(hdu_data_list[0], vmin=0, vmax=1e-17)

# plt.scatter(ra_pix_list[3], dec_pix_list[3], color='red') 
plt.scatter(ra_pix_list, dec_pix_list, color='red') 

# Measuring wavelength and opening angle

In [ ]:
#wavelength
#separation's .value method outputs value in degrees, have to convert to radians...
print(pos_list[0].separation(pos_list[2]))
print(pos_list[1].separation(pos_list[3]))

lam_helix = np.pi / 180. * np.mean([pos_list[0].separation(pos_list[2]).value, pos_list[1].separation(pos_list[3]).value])
print(lam_helix)

#opening angle, here we care about vertex ind = 3 and two points ind=0 , 1 
#then we apply dot product...
a = (ra_pix_list[3] - ra_pix_list[0], dec_pix_list[3] - dec_pix_list[0])
b = (ra_pix_list[3] - ra_pix_list[1], dec_pix_list[3] - dec_pix_list[1])

full_opening_rad = np.arccos(np.dot(a, b) / (np.sqrt(np.dot(a,a)) * np.sqrt(np.dot(b,b))))

print(full_opening_rad / 2. * 180/2/np.pi) #to check, printing a half opening angle in degrees

# Deriving precessional parameters from binary system

In [ ]:
#https://iopscience.iop.org/article/10.3847/1538-4357/ac7464/pdf
#this gives 
semi_major = 40/2. #au or 0.2" according to Trinidad 09 ~ 80 au ?
m_pri = 1.475 #msun
ma_mb = np.mean([1.3, 2.5]) #secondary/primary, according to cheng et al 2022 (link above), given as m_pri / m_sec
m_sec = m_pri / ma_mb
m_sum = m_pri + m_sec
mu = m_pri / m_sum
tau_orbit = np.sqrt(semi_major**3 / m_sum) #years

'''
tau_prec / np.cos(full_opening_rad/2) = 15/32 * mu/np.sqrt(1-mu) * sigma**1.5 / tau_orbit

tau_prec = np.sqrt( lam_helix * dist / (vtan * np.cos(full_opening_rad/2)) )

np.sqrt( lam_helix * dist / (vtan * np.cos(full_opening_rad/2)) ) 
    = 15/32 * mu/np.sqrt(1-mu) * sigma**1.5 / tau_orbit * np.cos(full_opening_rad/2) 
    
lam_helix * dist / (vtan * np.cos(full_opening_rad/2))  
    = (15/32 * mu/np.sqrt(1-mu) * sigma**1.5 / tau_orbit * np.cos(full_opening_rad/2) )**2
    
lam_helix / np.cos(full_opening_rad/2)**3
    = (15/32 * mu/np.sqrt(1-mu) * sigma**1.5 / tau_orbit )**2 * vtan / dist
'''

In [ ]:
m_pri, m_sec, m_sum, mu

In [ ]:
dist = 1.327e19 #430 pc -> m
vtan = 400 * 1000 #km/s -> m/s
sigma = 1/3

full_opening_rad = 20 * np.pi / 180 #degrees to rad

lam_helix = (15/32 * mu/np.sqrt(1-mu) * sigma**1.5 / tau_orbit/3.154e7 )**2 * vtan / dist * np.cos(full_opening_rad/2)**3

In [ ]:
lam_helix

# Applying the sinusoidal formula from Anglada 07 Sec 4.1.1

In [ ]:
#plotting resulting image
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import ZScaleInterval, ImageNormalize
from astropy.visualization.stretch import SinhStretch, LogStretch
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
import matplotlib.ticker
from matplotlib.ticker import LogFormatter, LogLocator, FixedLocator, FixedFormatter
                
#minor formatting for ticks
# plt.rcParams['xtick.labeltop'] = plt.rcParams['xtick.labelright'] = False
plt.rcParams['xtick.major.size'] = 14
plt.rcParams['ytick.major.size'] = 14

#best fit to a single sine curve

fig, ax = plt.subplots(figsize=(20, 20))
wcs = WCS(hdu_header_list[0])
ax = plt.subplot(projection=wcs)

#plotting data
plt.scatter(ra_pix_list, dec_pix_list, color='black') 

#treat x as the vertical units and y as the horizontal units in current image config...
# y_arr = np.linspace(-750, 750, 50)
# y0 = 1500 # lam_helix * np.pi * 2 # ra_pix_list[3]
# x0 = dec_pix_list[0]+50
# x = -y_arr * np.tan(full_opening_rad/2) * np.cos(2.*np.pi/lam_helix * (np.abs(y_arr) - y0))
# # x = y_arr*2 * np.tan(15*np.pi/180/2) * np.cos(2*np.pi*1.5 * (np.abs(y_arr) - int(y0)))
# # x = (y_arr+lam_helix / (2.*np.pi)) * np.tan(full_opening_rad / 2.) * np.cos(2.*np.pi/lam_helix * (np.abs(y_arr) - int(y0)))
# ax.plot(y_arr+y0, x+x0, color='red')

#plotting best fit 
from scipy.optimize import curve_fit

def y_precess(x, amp_prec, amp_orb, lam_prec, lam_orb, x0, y0):
        return amp_prec * np.sin(2*np.pi / lam_prec * (np.abs(x) - x0)) + \
                amp_orb * np.sin(2*np.pi / lam_orb * (np.abs(x) - x0)) + y0 

guess_amp_prec, guess_amp_orb, guess_lam_prec, guess_lam_orb, guess_x0, guess_y0 = \
    100, 10, 1000, 50, 1000, 1000
guess = [guess_amp_prec, guess_amp_orb, guess_lam_prec, guess_lam_orb, guess_x0, guess_y0]
params, cov = curve_fit(y_precess, ra_pix_list, dec_pix_list, p0=guess)
amp_prec, amp_orb, lam_prec, lam_orb, x0, y0 = params

dist_interp = np.linspace(np.min(ra_pix_list), np.max(ra_pix_list), 500) #plotting
ax.plot(dist_interp, y_precess(dist_interp, amp_prec, amp_orb, lam_prec, lam_orb, x0, y0), 'k--')

#plotting image
norm = ImageNormalize(stretch=LogStretch(), vmin=1e-18, vmax=100e-18)#vmin=data_interval[0], vmax=data_interval[1])
im = ax.imshow(hdu_data_list[0], norm=norm, cmap='tab20b') # plotting image for reference

#general formatting
ax.coords.grid(True, color='white', ls='solid', linewidth=0.75) #adding gridlines
ax.coords[0].set_axislabel('Right Ascension (J2000)', fontsize=30)
ax.coords[1].set_axislabel('Declination (J2000)', fontsize=30)   
ax.tick_params(axis='x', labelbottom=True, labeltop=False, labelright=False)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.invert_yaxis() #done because it's nicer, with declination increasing
ax.invert_xaxis() #done because it's nicer with ra decreasing

#colorbar, see 3rd answer from https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
cbar = plt.colorbar(im, cax=cax) 

#color bar label and tick labels
sub_labels = [0.1, 0.25, 10]
cbar.locator = LogLocator(base=10, subs=sub_labels)
cbar.ax.tick_params(labelsize=20)
cbar.update_ticks()
cbar.ax.yaxis.get_offset_text().set_fontsize(0)
cbar.set_label(label=r'$\rm Intensity~(x~{10}^{-18}~erg/s/{cm}^{2}/pix)$', size=20)
# cbar_tickfont = [cbar.ax.set_yticklabels(labels=cbar.ax.get_yticklabels())[i].set_fontweight('normal') \
#                      for i in range(len(cbar.ax.get_yticklabels()))]

plt.savefig('hops_361c_precession.png', dpi=300)
plt.savefig('hops_361c_precession.pdf', dpi=300)

In [ ]:
#pc, pc, pix
#amp_prec, amp_orb, lam_prec, lam_orb, x0, y0
print(amp_prec, lam_prec*hst_pixsize, lam_prec*hst_pixsize*430/206265)

print(amp_orb, lam_orb*hst_pixsize, lam_orb*hst_pixsize*430/206265)

print(x0, y0)

# Alternatively, use fit params to extract binary params...

From this, we can get the precession period given v_jet

Then we need to get sigma and the mass ratio (and/or compare with the literature) under assumptions
(close binary, distant blob of mass, etc)

In [ ]:
dist = 1.327e19 #430 pc -> m
vtan = 400 * 1000 #km/s -> m/s

tau_prec = np.sqrt( lam_helix * dist / (vtan * np.cos(full_opening_rad/2)) )

#sigma is rd / a, can perhaps use 1/3
sigma = 1./2.

#orbital period and/or orbital radius are then found as a function of mu, which we technically know
#https://iopscience.iop.org/article/10.3847/1538-4357/ac7464/pdf
#can alternatively use Trinidad 2009
# mu = ... np.mean([1.3, 2.5]) #secondary/primary, according to cheng et al 2022 (link above), given as mpri/m_sum
tau_orbit = tau_prec * 15/32 * mu/np.sqrt(1-mu) * sigma**1.5 *np.cos(full_opening_rad/2)

print(tau_prec, 'seconds?', mu, 'mpri/msum', tau_orbit, 'seconds?')